<a href="https://colab.research.google.com/github/coldsober-irene/hackathon1/blob/main/Copy_of_hackathon1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  -U easynmt

In [ ]:
# REQUIRED IMPORTS
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import json
import numpy as np
from easynmt import EasyNMT
import json # to access languages codes

In [2]:
# change the default limit of pandas columns to display
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **WEBSCRAPING SECTION** %%%%%%%%%%%%%%%%%%%%%

In [35]:
pd.set_option('display.max_columns', None)

In [3]:
# DATA MINING
class dataMining:
  def __init__(self):
    pass
    
  # SOUP
  def Soup(self, url):
    return bs(requests.get(url).content, "html.parser")
  
  def get_text(self):
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    parent = "https://www.jobinrwanda.com"
    co_links = {}
    app_link = []
    full_text = [[(p.get_text(), self.Soup(j_link).find("a", class_ = "btn btn-sm btn-success")) for p in dv.find_all('p', class_ = 'text-align-justify')] for j_link in [job_link for job_link in [parent+link['href'] if '/job/' in link['href'] else co_links.setdefault(link.text, parent+link['href']) 
    for div in soup.find_all('div', class_ = 'card-body p-2')[4] for link in div.find_all('a')] if '/job/' in job_link]
    for dv in self.Soup(j_link).find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
    ]
    return (full_text, app_link)

  def get_links(self):
    # GET SOUP OBJECT
    # soup = self.Soup(url = "https://www.jobinrwanda.com/jobs/all")
    soup = self.Soup(url = 'https://www.jobinrwanda.com/jobs/featured')
    # GET ALL JOBS LINKS
    company_infoLink = []
    
    all_jobsLinks = ["https://www.jobinrwanda.com"+link['href'] if "job" in link['href'] 
                     else company_infoLink.append(link.text) 
                     for div in soup.find_all("div", class_= "card-body p-2") 
                     for link in div.find_all("a")
                     ]
    all_jobsLinks = [link for link in all_jobsLinks if link]
    return (all_jobsLinks, company_infoLink)

  def get_jobInfo(self):
    jobLinks = self.get_links()[0]
    info = {}
    # REFINE THE INFO INTO A DICTIONARY
    def refine_info(ls):
      refined_info = {}
      infoZeroPattern = re.compile("\d+.*")
      views = infoZeroPattern.search(ls[0]).group()
      refined_info["views"] = views
      # OTHER DETAILS EXCEPT THE ONE ON POSITION 0
      otherInfoPattern = re.compile(":")
      for detail in ls[1:]:
        if detail.lower() == "apply":
          pass
        else:
          try:
            detailed = otherInfoPattern.split(detail)
            refined_info[detailed[0]] = detailed[1]
          except IndexError:
            pass
      return refined_info
       
    for index,url in enumerate(jobLinks):
      soup = self.Soup(url)
      pattern = re.compile("\s{2,}")
      job_info = [pattern.sub(" ",li.text.strip().replace("\n", "")) for ul in soup.find_all('ul', class_ = "list-group list-group-flush") 
      for li in ul.find_all('li')]
      # DICTIONANRY OF INFO
      info[index] = refine_info(job_info[:9])
    return info

  def get_description(self):
    jobLinks = self.get_links()[0]
    all_text= {}
    for index,link in enumerate(jobLinks):
      soup = self.Soup(link)
      # JOB TITLE
      job_title = soup.find('span', class_ = "field field--name-title field--type-string field--label-hidden")
      
      tags_content = soup.find_all('div', class_= "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
      
      for div in tags_content:
        temp = []
        try:
          # GET APPLICATION LINK
          app_link = soup.find("a", class_ = "btn btn-sm btn-success")['href']
          if app_link.startswith("/"):
            app_link = "https://www.jobinrwanda.com"+app_link
          temp.append(app_link)
          
        except TypeError:
          temp.append(np.nan)
        for tag in div.children:
          try:
            temp.append(tag.get_text())
          except (AttributeError, TypeError):
            pass
        temp.insert(0, job_title.text)
        all_text[index] = temp
    return all_text


# CLASS OBJECT CREATION
# jobInRwanda = dataMining()
# job_text = jobInRwanda.get_text()


In [4]:
# MERGE JOB DESCRIPTION WITH THE JOB INFO
def final_textDemo():
  global info
  # CLASS OBJECT CREATION
  jobInRwanda = dataMining()
  job_text = jobInRwanda.get_description()
  info = jobInRwanda.get_jobInfo()

  final_demo = []
  for index in list(info.keys()):
    info_value = list(info[index].values())
    # ADD COMPANY HIRING
    co_info = jobInRwanda.get_links()[1]
    info_value.append(co_info[index])
    # INCLUDE JOB TITLE
    info_value.append(job_text[index][0])
    # INCLUDE APPLY LINK
    info_value.append(job_text[index][1])
    # JOIN PARAGRAPHS 
    t = '\n'.join(job_text[index][2:])
    g = re.compile("\n|\\xa0").sub("", t)
    info_value.append(g)
    final_demo.append(info_value)
  return final_demo
# FUNCTION CALL
final_content = final_textDemo()

In [15]:
# FORM DATAFRAME
titles = list(info[0].keys()) + ["Company Hiring",'Job Title', "Application link", "Job Description"]
len(final_content[0])
# , titles
df = pd.DataFrame(data = final_content, columns = titles)


In [16]:
df.tail(30)

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Company Hiring,Job Title,Application link,Job Description
146,1224 times,"Kigali , RW",Administration,Bachelor,No experience,Part-time,"Friday, 18/11/2022 23",1,Embassy of the Federal Republic of Germany,"Assistant in the Press Department, Administrat...",https://www.jobinrwanda.com/form/default-job-a...,Summary:treatment of visa applicationspreparat...
147,370 times,"Kigali , RW",Other,Not specified,Not specified,Tender,"Friday, 11/11/2022 23",1,GIZ Rwanda,Expression of Interest (EOI)-Office Flooring W...,NaN,Expression of Interest (EOI)Office Flooring Wo...
148,406 times,", RW",Other,Not specified,Not specified,Tender,"Friday, 11/11/2022 23",1,GIZ Rwanda,Expression of Interest (EOI)-Office Partitioni...,NaN,Expression of Interest (EOI)Office Partitionin...
149,139 times,", RW",Other,,Tender,"Monday, 14/11/2022 17",1,Voluntary Service Overseas (VSO),Provision of Fuel (Petrol And Diesel) to Volun...,NaN,ADVERT TEMPLATE - EXPRESSION OF INTEREST (EOI)...,None
150,522 times,", RW",Other,Not specified,Not specified,Tender,"Saturday, 19/11/2022 10",1,Muganga SACCO,"Development of Mobile Banking Application, Imp...",NaN,INVITATION TO TENDERNo 009/T.MSC/2022/MSACCOMU...
151,1755 times,"Buhinga , RW","Agriculture, Agronomy, Demography and data an...",,Full-time,"Saturday, 19/11/2022 23",1,One Acre Fund,Rwanda Warehouse Manager,https://grnh.se/215948fc1us,"About One Acre FundFounded in 2006, One Acre F...",None
152,105 times,", RW",Other,Not specified,Not specified,Tender,"Sunday, 04/12/2022 11",1,ActionAid Rwanda (AAR),Request for Expression of Interest(EOI) for Pr...,NaN,REQUEST FOR EXPRESSION OF INTEREST FOR PRE‐QUA...
153,234 times,", RW",Other,Not specified,Not specified,Tender,"Sunday, 04/12/2022 11",1,ActionAid Rwanda (AAR),Request for Expression of Interest(EOI) for Pr...,NaN,REQUEST FOR EXPRESSION OF INTEREST FOR PRE‐QUA...
154,670 times,"Kigali , RW",Other,Bachelor,Mid career (3 to 5 years of experience),Full-time,"Monday, 21/11/2022 23",1,C Quest Capital,CSAT Officer – Rwanda,https://cquestcapital.bamboohr.com/jobs/view.p...,Job DescriptionCSAT Officer – RwandaCSAT Kigal...
155,587 times,", RW",Other,Not specified,Not specified,Contract,"Friday, 11/11/2022 17",1,RTI International,Request for Quote/Proposal (RFQ/RFP)- Pre-Qual...,NaN,Request for Quote/Proposal (RFQ/RFP)Commodity/...


# GET IT & COMPUTER RELATED JOBS, INTERNERSHIPS AND CONSULTANCY

In [ ]:
df2 = df.copy()
df2['Is-IT-Job'] = False
def isIT_Job():
  for index in range(len(df)):
    # print("row: ", "Computer" in row[2] or "IT" in row[2])
      if "Computer" in df2.iloc[index]['Sector'] or "IT" in df2.iloc[index]['Sector']:
        df2['Is-IT-Job'][index] = True

isIT_Job()

In [24]:
df2[df2['Is-IT-Job'] == True]

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Company Hiring,Job Title,Application link,Job Description,Is-IT-Job
26,160 times,"Kigali , RW","Computer and IT, Management, Project management",Bachelor,Senior (5+ years of experience),Full-time,"Tuesday, 10/01/2023 23",1,Ampersand Rwanda Ltd,Product Manager,https://ampersandelectricmobility.teamtailor.c...,Do you want to do work that matters? Do you wa...,True
120,1426 times,"Kigali , RW","Computer and IT, Other, Project management",Bachelor,Senior (5+ years of experience),Full-time,"Friday, 18/11/2022 23",1,CHANCEN International Rwanda,IT and MIS Director,https://www.jobinrwanda.com/form/default-job-a...,CHANCEN International is a non-profit organiza...,True
125,335 times,"Kigali , RW","Agriculture, Agronomy, Business, Computer and...",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Wednesday, 25/01/2023 23",1,One Acre Fund,Global Remote Sensing Manager,https://grnh.se/ed47136c1us,"About One Acre FundFounded in 2006, One Acre F...",True
136,772 times,", RW","Computer and IT, Education, Other",Bachelor,Not specified,Full-time,"Monday, 28/11/2022 23",1,University of Global Health Equity (UGHE),Instructional Technology Assistant,https://www.pih.org/pages/employment?p=job%2Fo...,DescriptionJob Title:Instructional Technology ...,True
173,2703 times,"Kigali , RW","Agriculture, Business, Computer and IT, Manag...",Not specified,Not specified,Full-time,"Tuesday, 06/12/2022 23",1,One Acre Fund,IT Operations Senior Manager,https://grnh.se/3041e7da1us,"About One Acre FundFounded in 2006, One Acre F...",True


In [ ]:
# DATA PRESENTATION
ITkeywords = ["Computer and IT", ""]

%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% **TRANSLATOR SECTION** %%%%%%%%%%%%%%%%%%%%%

In [ ]:
class translator:
  def __init__(self):
    # INITIALIZE THE MODEL FOR TRANSLATE
    self.model = EasyNMT("opus-mt")
  
  def supportedLangs(self, all_langs, supported):
    with open(all_langs) as f_json:
      content = f_json.read()
    langs = json.loads(content)
    print("langs: ", langs)

    with open(supported) as f2_json:
          content = f2_json.read()
    supplangs = json.loads(content)
    supp_codes = set(list(langs.values())).intersection(set(supplangs))
    supp_keyValue = {}
    for key in langs.keys():
      if langs[key] in supp_codes:
        supp_keyValue[key] = langs[key]
    return supp_keyValue
    

  def translate(self, sentences, targetLang, source = 'en', single = True):
    fail_message = "[FAIL]: destination lang is not a suported language by the model"
    success = 0
    if single:
      try:
        translation = self.model.translate(sentences, source_lang = source, target_lang = targetLang)
        return translation
      except:
        print(fail_message)
    else:
      translated = []
      for target in targetLang:
        try:
          translation = self.model.translate(sentences, source_lang = source, target_lang = target)
          translated.append(translation)
          success += 1
        except:
          print(fail_message)
      return translated, success


nest_asyncio : helps to make fastapi asynchronous

In [ ]:
# draft for consultancy
link = "https://www.jobinrwanda.com/jobs/consultancy"
soups = []
tags_contents = [[(child,soups.append(soup)) for child in final_div.children] for soup in [bs(requests.get(link).content, "html.parser") for link in ["https://jobinrwanda.com"+a['href'] 
        for div in soup.find_all('div', class_ = "card-body p-2") for a in div.find_all('a') if "job" in a['href']]]
         for final_div in soup.find_all('div', class_ = "clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item")
         ]



In [ ]:
c = "https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do"
s = bs(requests.get(c).content, 'html.parser')
print(s.prettify())

In [12]:
table = s.find('table', class_ = "article_table mb10")

In [ ]:
titles = table.thead.text
heads = [re.compile('[\r|\n|\t]+').sub('#', head.text.strip()) for head in table.find_all('thead')]
heads = heads[0].split('#')
heads

In [26]:
rows = table.find_all('tr')

In [30]:
full = []
for row in rows:
  temp = []
  for detail in row.find_all('td'):
    temp.append(detail.text.strip())
  full.append(temp[1:])
  # print("*"*50)

In [31]:
umucyo = pd.DataFrame(full, columns = heads)

In [32]:
umucyo

,Tender Name,Tender No,Status,Advertising Date,Deadline of Submitting,Planed Open Date,Stage Type
0,None,None,None,None,None,None,None
1,Hiring consultant Urban Mobility (Public trans...,000008/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/2022 10:00,27/12/2022 10:30,one stage
2,Hiring consultant Transport economist to suppo...,000009/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/2022 10:00,27/12/2022 10:30,one stage
3,science kits and laboratory equipment,000001/G/ICB/2022/2023/REB,Published,03/11/2022,20/12/2022 10:00,20/12/2022 10:30,one stage
4,Hiring an international Expert in Data Process...,000008/C/ICB/2022/2023/NISR,Published,02/11/2022,19/12/2022 10:00,19/12/2022 10:15,one stage
5,TENDER FOR OFFICE SUPPLIES AND STATIONARY,000001/G/NCB/2022/2023/Kiziguro,Published,04/11/2022,16/12/2022 10:00,16/12/2022 10:30,one stage
6,supply of labo kits reagents and consumables,000034/G/NCB/2022/2023/RAB,Published,10/11/2022,15/12/2022 11:00,15/12/2022 11:30,one stage
7,"Rehabilitation of administration bloc, constru...",000001/W/NCB/2022/2023/NIDA,Published,09/11/2022,15/12/2022 10:00,15/12/2022 10:30,one stage
8,Consultancy services in support of upgrading t...,000007/C/ICB/2022/2023/RDAP,Published,28/10/2022,14/12/2022 10:00,14/12/2022 10:30,1/2 stage
9,Hiring consultant Traffic signal engineer to s...,000013/C/NCB/2022/2023/CoK,Published,10/11/2022,14/12/2022 10:00,14/12/2022 10:30,one stage
